In [6]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import playerdashboardbyteamperformance
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamdashboardbyteamperformance
from nba_api.stats.endpoints import playerdashboardbylastngames
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.endpoints import teamdashboardbylastngames
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas as pd
import inspect
import numpy as np
import csv
import sys

headers = { 'Host': 'stats.nba.com', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0', 'Accept': 'application/json, text/plain, /', 'Accept-Language': 'en-US,en;q=0.5', 'Referer': 'https://stats.nba.com/', 'Accept-Encoding': 'gzip, deflate, br', 'Connection': 'keep-alive', }

In [18]:
#Player Input

player1_input = input('Enter player 1: ')
print(player1_input)

player1 = players.find_players_by_full_name(player1_input)[0]['id']
player1_name = players.find_players_by_full_name(player1_input)[0]['full_name']

player_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=player1, 
                                                                      season='2018-19', headers=headers)

# player_headers = player_info.by_year_player_dashboard.get_dict().get('headers')
# player_data = player_info.overall_player_dashboard.get_dict().get('data')[0]

# #last 20 games
player_headers = player_info.last20_player_dashboard.get_dict().get('headers')
player_data = player_info.last20_player_dashboard.get_dict().get('data')[0]

print(player_headers[5], player_data[5], 
	player_headers[2], player_data[2], 
	player_headers[26], player_data[26],
	player_headers[27], player_data[27])

Enter player 1: lebron james
lebron james
W_PCT 0.35 GP 20 PTS 553 PLUS_MINUS 19


In [19]:
#Team Input

team1_input = input('Enter team 1: ')
print(team1_input)
team1 = teams.find_teams_by_full_name(team1_input)[0]['id']

teamdashboard = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2018-19', last_n_games=1, 
                                                                   headers=headers)
teamdashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2018-19', 
                                                                    measure_type_detailed_defense='Advanced', 
                                                                        last_n_games=1, headers=headers)

##last n games
team_headers_basic = teamdashboard.overall_team_dashboard.get_dict().get('headers')
team_data_basic = teamdashboard.overall_team_dashboard.get_dict().get('data')[0]
team_headers = teamdashboard_adv.overall_team_dashboard.get_dict().get('headers')
team_data = teamdashboard_adv.overall_team_dashboard.get_dict().get('data')[0]

##last 20 games
# team_headers_basic = teamdashboard.last20_team_dashboard.get_dict().get('headers')
# team_data_basic = teamdashboard.last20_team_dashboard.get_dict().get('data')[0]
# team_headers = teamdashboard_adv.last20_team_dashboard.get_dict().get('headers')
# team_data = teamdashboard_adv.last20_team_dashboard.get_dict().get('data')[0]

print(team1, team_headers, team_data)
# print(team1, team_headers[23], team_data[23], team_headers[8], team_data[8], team_headers[10], team_data[10], 
#       team_headers[2], team_data[2], team_headers[3], team_data[3], 
#       team_headers[4], team_data[4], team_headers[21], team_data[21], team_headers[6], team_data[6])

Enter team 1: lakers
lakers
1610612747 ['GROUP_SET', 'GROUP_VALUE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK', 'NET_RATING_RANK', 'AST_PCT_RANK', 'AST_TO_RANK', 'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK', 'REB_PCT_RANK', 'TM_TOV_PCT_RANK', 'EFG_PCT_RANK', 'TS_PCT_RANK', 'PACE_RANK', 'PIE_RANK', 'CFID', 'CFPARAMS'] ['Overall', '2018-19', 1, 0, 1, 0.0, 48.0, 99.5, 102.0, 101.3, 105.1, -1.8, -3.0, 0.725, 1.71, 19.5, 0.393, 0.704, 0.545, 0.172, 0.479, 0.498, 102.1, 99.0, 82.5, 99, 0.486, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 163, '2018-19']


In [20]:
roster1 = teamplayerdashboard.TeamPlayerDashboard(team_id=team1, last_n_games=1, 
                                                  season='2018-19', headers=headers).get_dict()#.PlayersSeasonTotals(team_id=team1, last_n_games=20)
team1_roster_last20 = []
team1_roster_lastn = []
#print(roster1)['resultSets'][1])#['rowSet'])
for n in roster1['resultSets'][1]['rowSet']:
    print(n[2], 'GP:', n[3], 'id:', n[1], 'Min', n[7])
    team1_roster_lastn.append(n[2])
    
print(team1_roster_lastn)
#(PER * MP)/n_players

Alex Caruso GP: 1 id: 1627936 Min 40.56666666666667
Isaac Bonga GP: 1 id: 1629067 Min 9.08
JaVale McGee GP: 1 id: 201580 Min 19.416666666666668
Jemerrio Jones GP: 1 id: 1629203 Min 46.483333333333334
Johnathan Williams GP: 1 id: 1629140 Min 28.583333333333332
Kentavious Caldwell-Pope GP: 1 id: 203484 Min 43.95
Mike Muscala GP: 1 id: 203488 Min 35.403333333333336
Moritz Wagner GP: 1 id: 1629021 Min 16.516666666666666
['Alex Caruso', 'Isaac Bonga', 'JaVale McGee', 'Jemerrio Jones', 'Johnathan Williams', 'Kentavious Caldwell-Pope', 'Mike Muscala', 'Moritz Wagner']


In [114]:
def off_per_calculator(player, team):
    
    agg_PER = 0 
    agg_adj_PER = 0
    num_players = 0
    PER_MP = 0
    agg_min = 0
    
    for n in player:
        p_id = players.find_players_by_full_name(n)[0]['id']
        p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=p_id, season='2018-19',
                                                                        last_n_games=1, headers=headers)
        
        ##last 20 games        
#         player_headers = p_info.last20_player_dashboard.get_dict().get('headers')
#         player_data = p_info.last20_player_dashboard.get_dict().get('data')[0]
        
        player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
        player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]

 
    #####PER player stats####
        MP = player_data[6]
        ThrP = player_data[10] 
        AST = player_data[19]
        TOV = player_data[20]
        FGA = player_data[8]
        FG = player_data[7] + player_data[10] 
        FTA = player_data[14]
        FT = player_data[13]
        TRB = player_data[18]
        ORB = player_data[16]
        STL = player_data[21]
        BLK = player_data[22]
        PF = player_data[24]

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = team_data_basic[19]
        team_FG = team_data_basic[7]+team_data_basic[10]
        team_pace = team_data[23] 
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)
        
        agg_PER += PER_calc_player 
        agg_adj_PER += adj_PER_calc_player
        num_players +=1
        PER_MP += (adj_PER_calc_player * MP)
        PER_MP_n = PER_MP/num_players
        
        print(n, 'Min', MP, 'adj_PER_calc_player', adj_PER_calc_player, 'PER*MP_Player', (adj_PER_calc_player*MP))
        
    print('Agg_PER * MP', PER_MP)
    
    print('PERxMP/n', PER_MP_n, 'Total PER', agg_PER, 'Total adj_PER', agg_adj_PER)
    print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)
#    print('league stats:', factor, VOP, DRB_perc)

In [115]:
off_per_calculator(team1_roster_lastn, team1)

Danilo Gallinari Min 24.8 adj_PER_calc_player 0.426271056923702 PER*MP_Player 10.57152221170781
Garrett Temple Min 21.583333333333332 adj_PER_calc_player 0.3514380730823768 PER*MP_Player 7.585205077361299
Ivica Zubac Min 29.783333333333335 adj_PER_calc_player 0.6585686980390959 PER*MP_Player 19.61437105659774
JaMychal Green Min 25.333333333333332 adj_PER_calc_player 0.5061284203324381 PER*MP_Player 12.821919981755098
Jerome Robinson Min 16.45 adj_PER_calc_player 0.22087037238040508 PER*MP_Player 3.6333176256576634
Landry Shamet Min 29.416666666666668 adj_PER_calc_player 0.5033830669166712 PER*MP_Player 14.807851885132079
Lou Williams Min 23.316666666666666 adj_PER_calc_player 0.3062138270859272 PER*MP_Player 7.139885734886869
Montrezl Harrell Min 20.816666666666666 adj_PER_calc_player 0.9940717283930562 PER*MP_Player 20.693259812715453
Patrick Beverley Min 21.316666666666666 adj_PER_calc_player 1.013862675943175 PER*MP_Player 21.612172708855347
Shai Gilgeous-Alexander Min 24.1 adj_PER_

In [21]:
#Alternate Offensive Rating

def off_per_calc_alt(player, team):
    
    agg_off_rating = 0
    agg_min = 0

    player_info_off = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=5, season='2018-19', 
                                                                  measure_type_detailed_defense='Advanced', 
                                                                  headers=headers)

    player_off_headers = player_info_off.league_dash_player_stats.get_dict().get('headers')
    player_off_data = player_info_off.league_dash_player_stats.get_dict().get('data')

#     print('headers:', player_off_headers)
    #print('data:', player_def_data)

    for p in team1_roster_lastn:
        for n in player_off_data:
            if p in n:
                print(n[1], '\n', 'E_Off_Rating', n[10], 'Min', n[9])
                agg_off_rating += (n[10]*n[9])
                agg_min += n[9]

    avg_off_rating = agg_off_rating/len(team1_roster_lastn)

    print('agg_off_rating', agg_off_rating, 'avg_off_rating', avg_off_rating, 'agg_min', agg_min, 
          'agg_off_rat_min', agg_off_rating/agg_min) 

In [107]:
#Individual Player Defense Stats

def def_per_calc(team_roster):
    agg_def_rating = 0
    agg_min = 0

    # player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=1, season='2018-19', 
    #                                                                   measure_type_detailed_defense='Defense')
    player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(opponent_team_id=, month=4, season='2018-19', 
                                                                      measure_type_detailed_defense='Defense',
                                                                 headers=headers)



    player_def_headers = player_info_def.league_dash_player_stats.get_dict().get('headers')
    player_def_data = player_info_def.league_dash_player_stats.get_dict().get('data')

    print('headers:', player_def_headers)
        #print('data:', player_def_data)

    for p in team1_roster_lastn:
        for n in player_def_data:
            if p in n:
                print(n[1], '\n', 'Def Rating', n[10], 'Min', n[9])
                agg_def_rating += (n[10]*n[9])
                agg_min += n[9]


    avg_def_rating = agg_def_rating/len(team1_roster)
    avg_def_rating_min = agg_def_rating/agg_min

    print('agg_def_rating', agg_def_rating, 'avg_def_rating', avg_def_rating, 'agg_min', agg_min) 
    print('avg_def_rating_min', avg_def_rating_min)

In [4]:
def off_per_calculator_pandas(player, team):
    
    agg_PER = 0 
    agg_adj_PER = 0
    num_players = 0
    PER_MP = 0
    agg_min = 0

    
    for n in player:
#         p_id = players.find_players_by_full_name(n)[0]['id']
#         p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=p_id, season='2018-19',
#                                                                         last_n_games=1)
        
#         ##last 20 games        
# #         player_headers = p_info.last20_player_dashboard.get_dict().get('headers')
# #         player_data = p_info.last20_player_dashboard.get_dict().get('data')[0]
        
#         player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
#         player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]

 
    #####PER player stats####
        MP = team1_roster['MINUTES']
        ThrP = team1_roster['3PMa'] 
        AST = team1_roster['AST']
        TOV = team1_roster['TOV']
        FGA = team1_roster['FGA']
        FG = team1_roster['FGM'] 
        FTA = team1_roster['FTA']
        FT = team1_roster['FTM']
        TRB = team1_roster['REB']
        ORB = team1_roster['OREB']
        STL = team1_roster['STL']
        BLK = team1_roster['BLK']
        PF = team1_roster['PF']

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = 25.6
        team_FG = 42.6
        team_pace = 103.6
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)
        
        agg_PER += PER_calc_player 
        agg_adj_PER += adj_PER_calc_player
        num_players +=1
        PER_MP = adj_PER_calc_player * MP
        PER_MP_n += PER_MP
        
        print(n, 'Min', MP, 'adj_PER_calc_player', adj_PER_calc_player, 'PER*MP_Player', (adj_PER_calc_player*MP))
        
    print('Agg_PER * MP_n', PER_MP)
    
#     print('PERxMP/n', PER_MP_n, 'Total PER', agg_PER, 'Total adj_PER', agg_adj_PER)
    print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)
#    print('league stats:', factor, VOP, DRB_perc)

In [98]:
data = pd.read_csv("NBA Merged Sheets - Sheet2.csv") 
data_t = pd.read_csv("team_stats_basic_2018-19.csv")
df = pd.read_csv("team 2018-19.csv") 

teams = ['ATL', 'IND', 'MIL', 'ORL', 'PHI', 'SAS', 'POR', 'NOP', 'UTA', 'HOU', 'BOS', 
         'MEM', 'BKN', 'MIA', 'CHI', 'MIN', 'PHX', 'DET', 'NYK', 'LAL', 'CHA', 'TOR', 
         'WAS', 'SAC', 'LAC', 'OKC', 'CLE', 'DAL', 'DEN', 'GSW']

In [99]:
def regression(team1):
    
    with open('regression_list.csv', 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)
        headers = ['Team1','T1_OR5', 'T1_DR5', 'Team2', 'T2_OR5', 'T2_DR5', 'Spread']
        writer.writerow(headers)
    csvFile.close() 
    
    team2 = []
    date = []

    score = []

    df2 = df[['Team', 'Matchup', 'Date', 'OffRtg', 'DefRtg', 'ASTRatio', 'PACE']]
    df2 = df[df['Team'] == team1]

    opp = []

    team2 = df2['Matchup']

    for row in team2:
        opp.append(row.replace(team1+'vs.', '').replace(team1+'@', ''))

    df2['Opp'] = opp

    team1_off_avg = 0
    team1_def_avg = 0

    team2_off_avg = 0
    team2_def_avg = 0

    for index, row in df2.head(n=77).iterrows(): 
        whole_row = []
        team2 = row['Opp']
        date = row['Date']
        df3 = df[df['Team'] == team2]

        score = data.loc[(data['DATE']==date) & (data['TEAM']==team1)]
        score = score.iloc[0]
        score = (score["team_pts"], score['team_plus_minus'])
        print('===', row['Team'], row['OffRtg'], row['DefRtg'], 
              row['Date'], row['Opp'], score, "===") 
                
        print(row['Team'], 'Last5_Off', row['Last5_Off'], 'Last5_Def', row['Last5_Def'])
        whole_row.append(row['Team'])
        whole_row.append(row['Last5_Off'])
        whole_row.append(row['Last5_Def'])

        df3 = df3.loc[df3['Date'] == date]
        for index, row in df3.iterrows():
            print(row['Team'], 'Last5_Off', row['Last5_Off'], 'Last5_Def', row['Last5_Def'], '\n')
            whole_row.append(row['Team'])
            whole_row.append(row['Last5_Off'])
            whole_row.append(row['Last5_Def'])
            whole_row.append(score[1])
            
            with open('regression_list.csv', 'a') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(whole_row)
            csvFile.close() 
    


In [100]:
for team in teams:
    regression(team)

/Users/stan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


=== ATL 122.9 125.0 04/10/19 IND (134, -1.0) ===
ATL Last5_Off 110.22 Last5_Def 117.62
IND Last5_Off 108.68 Last5_Def 110.44 

=== ATL 99.1 106.5 04/07/19 MIL (107, -8.0) ===
ATL Last5_Off 110.0 Last5_Def 119.68
MIL Last5_Off 118.16 Last5_Def 113.78 

=== ATL 102.7 134.2 04/05/19 ORL (113, -36.0) ===
ATL Last5_Off 112.46 Last5_Def 113.9
ORL Last5_Off 111.9 Last5_Def 113.6 

=== ATL 126.2 118.4 04/03/19 PHI (130, 8.0) ===
ATL Last5_Off 110.68 Last5_Def 113.52
PHI Last5_Off 110.46 Last5_Def 114.72 

=== ATL 108.8 113.6 04/02/19 SAS (111, -6.0) ===
ATL Last5_Off 111.0 Last5_Def 112.52
SAS Last5_Off 113.02 Last5_Def 111.18 

=== ATL 114.3 115.4 03/31/19 MIL (136, 1.0) ===
ATL Last5_Off 108.94 Last5_Def 113.4
MIL Last5_Off 114.66 Last5_Def 101.26 

=== ATL 98.0 116.8 03/29/19 POR (98, -20.0) ===
ATL Last5_Off 106.5 Last5_Def 109.1
POR Last5_Off 121.96 Last5_Def 112.64 

=== ATL 115.0 105.3 03/26/19 NOP (130, 10.0) ===
ATL Last5_Off 106.36 Last5_Def 112.62
NOP Last5_Off 103.12 Last5_Def 114.

OKC Last5_Off 102.74 Last5_Def 111.08 

=== IND 124.0 88.0 03/24/19 DEN (124, 36.0) ===
IND Last5_Off 101.72 Last5_Def 108.7
DEN Last5_Off 114.68 Last5_Def 107.32 

=== IND 88.1 109.8 03/21/19 GSW (89, -23.0) ===
IND Last5_Off 105.12 Last5_Def 107.16
GSW Last5_Off 110.84 Last5_Def 105.18 

=== IND 102.8 108.5 03/19/19 LAC (109, -6.0) ===
IND Last5_Off 102.72 Last5_Def 106.88
LAC Last5_Off 118.38 Last5_Def 114.1 

=== IND 104.3 111.6 03/18/19 POR (98, -8.0) ===
IND Last5_Off 101.46 Last5_Def 107.72
POR Last5_Off 115.1 Last5_Def 109.46 

=== IND 107.5 109.7 03/16/19 DEN (100, -2.0) ===
IND Last5_Off 101.6 Last5_Def 105.78
DEN Last5_Off 115.08 Last5_Def 109.82 

=== IND 105.9 103.9 03/14/19 OKC (108, 2.0) ===
IND Last5_Off 104.24 Last5_Def 109.9
OKC Last5_Off 108.1 Last5_Def 105.86 

=== IND 105.1 102.1 03/12/19 NYK (103, 5.0) ===
IND Last5_Off 108.38 Last5_Def 113.2
NYK Last5_Off 97.92 Last5_Def 108.88 

=== IND 90.8 107.1 03/10/19 PHI (89, -17.0) ===
IND Last5_Off 111.48 Last5_Def 114.4

CHA Last5_Off 116.88 Last5_Def 108.74 

=== MIL 111.5 101.9 01/21/19 DAL (116, 10.0) ===
MIL Last5_Off 113.94 Last5_Def 100.5
DAL Last5_Off 112.82 Last5_Def 114.76 

=== MIL 118.0 108.0 01/19/19 ORL (118, 10.0) ===
MIL Last5_Off 112.64 Last5_Def 99.66
ORL Last5_Off 111.4 Last5_Def 112.8 

=== MIL 106.7 98.1 01/16/19 MEM (111, 10.0) ===
MIL Last5_Off 114.34 Last5_Def 100.44
MEM Last5_Off 99.4 Last5_Def 110.12 

=== MIL 119.2 82.7 01/15/19 MIA (124, 38.0) ===
MIL Last5_Off 113.7 Last5_Def 108.26
MIA Last5_Off 108.18 Last5_Def 109.24 

=== MIL 119.8 101.8 01/13/19 ATL (133, 19.0) ===
MIL Last5_Off 115.22 Last5_Def 107.72
ATL Last5_Off 104.98 Last5_Def 108.2 

=== MIL 106.0 111.9 01/11/19 WAS (106, -7.0) ===
MIL Last5_Off 119.22 Last5_Def 105.76
WAS Last5_Off 112.26 Last5_Def 106.64 

=== MIL 111.5 103.8 01/09/19 HOU (116, 7.0) ===
MIL Last5_Off 121.96 Last5_Def 107.12
HOU Last5_Off 117.52 Last5_Def 113.6 

=== MIL 115.2 102.0 01/07/19 UTA (114, 12.0) ===
MIL Last5_Off 121.32 Last5_Def 106

ORL Last5_Off 115.4 Last5_Def 111.28
DEN Last5_Off 111.46 Last5_Def 108.42 

=== ORL 93.8 96.9 11/20/18 TOR (91, -2.0) ===
ORL Last5_Off 118.76 Last5_Def 109.02
TOR Last5_Off 110.62 Last5_Def 105.32 

=== ORL 133.7 118.2 11/18/18 NYK (131, 14.0) ===
ORL Last5_Off 115.42 Last5_Def 106.76
NYK Last5_Off 103.16 Last5_Def 115.9 

=== ORL 125.0 111.4 11/17/18 LAL (130, 13.0) ===
ORL Last5_Off 109.06 Last5_Def 104.28
LAL Last5_Off 107.8 Last5_Def 104.02 

=== ORL 113.3 109.3 11/14/18 PHI (111, 5.0) ===
ORL Last5_Off 107.66 Last5_Def 103.04
PHI Last5_Off 107.24 Last5_Def 109.0 

=== ORL 111.2 120.6 11/12/18 WAS (109, -8.0) ===
ORL Last5_Off 109.54 Last5_Def 101.6
WAS Last5_Off 106.64 Last5_Def 113.94 

=== ORL 110.6 85.6 11/11/18 NYK (115, 26.0) ===
ORL Last5_Off 106.8 Last5_Def 108.72
NYK Last5_Off 107.34 Last5_Def 109.2 

=== ORL 117.0 106.9 11/09/18 WAS (117, 9.0) ===
ORL Last5_Off 102.44 Last5_Def 107.92
WAS Last5_Off 101.6 Last5_Def 116.24 

=== ORL 93.2 99.0 11/07/18 DET (96, -7.0) ===
O

...

DAL Last5_Off 104.74 Last5_Def 103.94 

=== SAC 108.7 112.1 12/14/18 GSW (125, -5.0) ===
SAC Last5_Off 119.32 Last5_Def 108.5
GSW Last5_Off 112.36 Last5_Def 104.02 

=== SAC 139.6 130.0 12/12/18 MIN (141, 11.0) ===
SAC Last5_Off 112.14 Last5_Def 103.26
MIN Last5_Off 108.36 Last5_Def 107.76 

=== SAC 108.0 89.0 12/10/18 CHI (108, 19.0) ===
SAC Last5_Off 114.26 Last5_Def 111.8
CHI Last5_Off 94.2 Last5_Def 113.54 

=== SAC 99.0 110.3 12/08/18 IND (97, -10.0) ===
SAC Last5_Off 114.64 Last5_Def 113.7
IND Last5_Off 103.38 Last5_Def 99.14 

=== SAC 131.6 112.2 12/07/18 CLE (129, 19.0) ===
SAC Last5_Off 110.42 Last5_Def 113.76
CLE Last5_Off 99.64 Last5_Def 117.6 

=== SAC 118.4 101.0 12/04/18 PHX (122, 17.0) ===
SAC Last5_Off 111.28 Last5_Def 115.78
PHX Last5_Off 99.3 Last5_Def 112.78 

=== SAC 103.7 103.8 12/01/18 IND (111, 1.0) ===
SAC Last5_Off 111.82 Last5_Def 115.38
IND Last5_Off 108.14 Last5_Def 107.8 

=== SAC 118.6 131.7 11/29/18 LAC (121, -12.0) ===
SAC Last5_Off 109.44 Last5_Def 114.

OKC Last5_Off 116.24 Last5_Def 112.36
HOU Last5_Off 127.8 Last5_Def 103.32 

=== OKC 128.2 123.5 04/07/19 MIN (132, 6.0) ===
OKC Last5_Off 112.9 Last5_Def 108.5
MIN Last5_Off 112.36 Last5_Def 115.28 

=== OKC 119.4 106.8 04/05/19 DET (123, 13.0) ===
OKC Last5_Off 109.42 Last5_Def 109.92
DET Last5_Off 103.34 Last5_Def 103.62 

=== OKC 116.7 101.0 04/02/19 LAL (119, 16.0) ===
OKC Last5_Off 109.76 Last5_Def 111.96
LAL Last5_Off 114.3 Last5_Def 104.8 

=== OKC 104.0 108.2 03/31/19 DAL (103, -3.0) ===
OKC Last5_Off 109.14 Last5_Def 112.28
DAL Last5_Off 114.48 Last5_Def 113.82 

=== OKC 112.9 122.3 03/29/19 DEN (105, -10.0) ===
OKC Last5_Off 107.96 Last5_Def 110.56
DEN Last5_Off 103.44 Last5_Def 110.16 

=== OKC 111.5 104.2 03/27/19 IND (107, 8.0) ===
OKC Last5_Off 102.74 Last5_Def 111.08
IND Last5_Off 105.34 Last5_Def 105.52 

=== OKC 102.0 113.9 03/25/19 MEM (103, -12.0) ===
OKC Last5_Off 103.12 Last5_Def 109.48
MEM Last5_Off 113.44 Last5_Def 121.96 

=== OKC 118.4 111.2 03/22/19 TOR (116,

=== CLE 102.1 116.8 02/02/19 DAL (98, -13.0) ===
CLE Last5_Off 103.38 Last5_Def 110.28
DAL Last5_Off 112.2 Last5_Def 106.18 

=== CLE 113.7 111.9 01/29/19 WAS (116, 3.0) ===
CLE Last5_Off 102.82 Last5_Def 115.16
WAS Last5_Off 107.3 Last5_Def 107.48 

=== CLE 109.5 105.2 01/27/19 CHI (104, 3.0) ===
CLE Last5_Off 100.14 Last5_Def 116.9
CHI Last5_Off 103.74 Last5_Def 114.78 

=== CLE 100.0 106.4 01/25/19 MIA (94, -6.0) ===
CLE Last5_Off 103.96 Last5_Def 123.06
MIA Last5_Off 101.08 Last5_Def 111.04 

=== CLE 103.0 121.8 01/23/19 BOS (103, -20.0) ===
CLE Last5_Off 103.76 Last5_Def 117.9
BOS Last5_Off 112.66 Last5_Def 107.68 

=== CLE 90.7 106.1 01/21/19 CHI (88, -16.0) ===
CLE Last5_Off 106.74 Last5_Def 123.04
CHI Last5_Off 104.48 Last5_Def 123.36 

=== CLE 110.9 136.3 01/19/19 DEN (102, -22.0) ===
CLE Last5_Off 107.96 Last5_Def 122.44
DEN Last5_Off 118.72 Last5_Def 115.82 

=== CLE 96.1 113.9 01/18/19 UTA (99, -16.0) ===
CLE Last5_Off 113.48 Last5_Def 125.56
UTA Last5_Off 108.98 Last5_Def 

=== DAL 103.1 78.4 12/10/18 ORL (101, 25.0) ===
DAL Last5_Off 105.72 Last5_Def 109.54
ORL Last5_Off 107.9 Last5_Def 107.36 

=== DAL 118.9 113.0 12/08/18 HOU (107, 3.0) ===
DAL Last5_Off 108.6 Last5_Def 109.92
HOU Last5_Off 112.14 Last5_Def 113.56 

=== DAL 100.0 123.4 12/05/18 NOP (106, -26.0) ===
DAL Last5_Off 111.66 Last5_Def 106.46
NOP Last5_Off 113.7 Last5_Def 112.66 

=== DAL 107.8 101.0 12/04/18 POR (111, 9.0) ===
DAL Last5_Off 113.66 Last5_Def 109.08
POR Last5_Off 110.86 Last5_Def 119.28 

=== DAL 106.5 103.8 12/02/18 LAC (114, 4.0) ===
DAL Last5_Off 110.7 Last5_Def 109.18
LAC Last5_Off 113.72 Last5_Def 107.2 

=== DAL 95.4 106.5 11/30/18 LAL (103, -11.0) ===
DAL Last5_Off 113.8 Last5_Def 109.68
LAL Last5_Off 98.4 Last5_Def 101.4 

=== DAL 133.3 114.9 11/28/18 HOU (128, 20.0) ===
DAL Last5_Off 110.74 Last5_Def 100.16
HOU Last5_Off 119.58 Last5_Def 119.6 

=== DAL 115.3 106.1 11/24/18 BOS (113, 9.0) ===
DAL Last5_Off 107.68 Last5_Def 97.6
BOS Last5_Off 104.72 Last5_Def 105.32 



GSW Last5_Off 112.76 Last5_Def 112.0
MEM Last5_Off 114.24 Last5_Def 116.22 

=== GSW 133.0 122.6 03/24/19 DET (121, 7.0) ===
GSW Last5_Off 107.52 Last5_Def 104.56
DET Last5_Off 120.56 Last5_Def 115.86 

=== GSW 95.8 131.3 03/23/19 DAL (91, -35.0) ===
GSW Last5_Off 111.66 Last5_Def 100.9
DAL Last5_Off 112.5 Last5_Def 116.46 

=== GSW 109.8 88.1 03/21/19 IND (112, 23.0) ===
GSW Last5_Off 110.84 Last5_Def 105.18
IND Last5_Off 105.12 Last5_Def 107.16 

=== GSW 115.8 105.9 03/19/19 MIN (117, 10.0) ===
GSW Last5_Off 110.92 Last5_Def 104.0
MIN Last5_Off 108.9 Last5_Def 118.08 

=== GSW 109.4 112.1 03/18/19 SAS (105, -6.0) ===
GSW Last5_Off 106.96 Last5_Def 105.74
SAS Last5_Off 112.74 Last5_Def 102.76 

=== GSW 106.8 85.4 03/16/19 OKC (110, 22.0) ===
GSW Last5_Off 107.82 Last5_Def 110.32
OKC Last5_Off 105.8 Last5_Def 101.84 

=== GSW 116.5 113.0 03/13/19 HOU (106, 2.0) ===
GSW Last5_Off 103.16 Last5_Def 107.52
HOU Last5_Off 112.02 Last5_Def 101.8 

=== GSW 105.7 109.5 03/10/19 PHX (111, -4.0) 

In [60]:
#Theory: If second game of back-to-back is a road game, odds of winning decrease 7-8%, multiply ratings by 0.93.